In [1]:
import pandas as pd

In [3]:
dataFrame = pd.read_csv("breast-cancer-wisconsin.data",header=None)

In [4]:
dataFrame.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [14]:
import numpy as np
dataFrame.replace({6: {'?': np.nan}}, regex=False,inplace=True)
dataFrame = dataFrame.dropna()

In [21]:
Predictors = dataFrame.iloc[:,1:len(dataFrame.columns)-1]
Predictors.head()

,1,2,3,4,5,6,7,8,9
0,5,1,1,1,2,1,3,1,1
1,5,4,4,5,7,10,3,2,1
2,3,1,1,1,2,2,3,1,1
3,6,8,8,1,3,4,3,7,1
4,4,1,1,3,2,1,3,1,1


In [26]:
Classes = dataFrame.iloc[:,10].map({4 : 1, 2 : 0})
Classes.head()

0    0
1    0
2    0
3    0
4    0
Name: 10, dtype: int64

In [27]:
from sklearn import preprocessing # Normalize Data
names = Predictors.columns
scaler = preprocessing.StandardScaler()
Predictors_norm = scaler.fit_transform(Predictors)
Predictors_norm = pd.DataFrame(Predictors_norm, columns=names)

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [29]:
Predictors_norm.head()

,1,2,3,4,5,6,7,8,9
0,0.197905,-0.702212,-0.741774,-0.639366,-0.555608,-0.698853,-0.181827,-0.612927,-0.3484
1,0.197905,0.277252,0.262783,0.758032,1.695166,1.772867,-0.181827,-0.285105,-0.3484
2,-0.511643,-0.702212,-0.741774,-0.639366,-0.555608,-0.424217,-0.181827,-0.612927,-0.3484
3,0.552679,1.583204,1.602192,-0.639366,-0.105454,0.125054,-0.181827,1.354008,-0.3484
4,-0.156869,-0.702212,-0.741774,0.059333,-0.555608,-0.698853,-0.181827,-0.612927,-0.3484


In [30]:
from sklearn.model_selection import train_test_split # Split data into test and train
X_train, X_test, y_train, y_test = train_test_split(Predictors_norm, Classes, test_size = 0.33, random_state = 42)

In [33]:
from sklearn.model_selection import GridSearchCV # Grid Search
from sklearn import svm # SVM Classifier
def SVM_Optimization(SVM_Predictors, SVM_Classes, num_of_folds):
    # SVM_Predictors: Feature matrix of the data
    # SVM_Classes   : Class (Label / Target) vector of the data
    # num_of_folds  : fold number for k fold - Cross Validation
    
    # Smaller tolerance = large margin, Larger tolerance = Small margin
    tolerances = [0.001, 0.01, 0.05, 0.1, 0.5, 1, 3, 5, 10]
    
    # Gamma is kind of variance. If Gamma is too large, it is more like overfitting
    Gammas = [0.001, 0.01, 0.05, 0.1, 0.5, 1]
    parameters_of_gridsearch = {'C' : tolerances, 'gamma' : Gammas, 'kernel' :
        ['linear', 'poly', 'rbf']}
    
    grid_search_optimization = GridSearchCV(svm.SVC(),
                                            parameters_of_gridsearch, 
                                            cv = num_of_folds, iid = False)
    grid_search_optimization.fit(SVM_Predictors, SVM_Classes)
    return grid_search_optimization.best_params_

In [34]:
optimized_param = SVM_Optimization(X_train, y_train,10)

In [35]:
svm_model = svm.SVC(C = optimized_param['C'], gamma = optimized_param['gamma'],
                    kernel = optimized_param['kernel'])

svm_model.fit(X_train, y_train)

SVC(C=5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [47]:
from sklearn.metrics import accuracy_score # Accuracy
from sklearn.model_selection import KFold # K-Fold Cross Validation

kfold = KFold(n_splits=10,random_state=7, shuffle=True)
scores = cross_val_score(svm_model, X_train, y_train, scoring='accuracy' ,cv=kfold).mean()

svm_train_acc = scores
print("SVM training Accuracy of the mean of 10 - Fold Cross Validation is ", svm_train_acc)

svm_test_acc = accuracy_score(y_test, svm_predictions)
print("SVM test Accuracy is ", svm_test_acc)

SVM training Accuracy of the mean of 10 - Fold Cross Validation is  0.9782608695652174
SVM test Accuracy is  0.9646017699115044


In [44]:
from sklearn.model_selection import GridSearchCV # Grid Search
from sklearn.ensemble import RandomForestClassifier # RF Classifier
# Random Forest Optimization
def RF_Optimization(RF_Predictors, RF_Classes, num_of_folds):
    # RF_Predictors: Feature matrix of the data
    # RF_Classes   : Class (Label / Target) vector of the data
    # num_of_folds : fold number for k fold - Cross Validation
    
    # Number of trees
    num_tree = [int (x) for x in np.linspace(10, 100, 10)]
    
    # Method for RF
    bootstrap = [True, False]
   
    parameters_of_gridsearch = {'n_estimators' : num_tree,
                                'bootstrap' : bootstrap,
                                'criterion' : ['gini', 'entropy']}
    
    kfold = KFold(n_splits=num_of_folds,random_state=7, shuffle=True)
    
    # GridSearch with k-fold Cross Validation
    grid_search_optimization = GridSearchCV(estimator = RandomForestClassifier(),
                                            param_grid = parameters_of_gridsearch, 
                                            cv = kfold, iid = False)
    grid_search_optimization.fit(RF_Predictors, RF_Classes)
    
    return grid_search_optimization.best_params_

In [46]:
# Random Forest Classification
from sklearn.model_selection import cross_val_score # K-Fold Cross Validation
from sklearn.model_selection import KFold # K-Fold Cross Validation

optimized_param_rf = RF_Optimization(X_train, y_train,10)

rf_model = RandomForestClassifier(n_estimators = optimized_param_rf['n_estimators'],
                                  bootstrap = optimized_param_rf['bootstrap'],
                                  criterion = optimized_param_rf['criterion'],
                                  random_state = 42)
rf_model.fit(X_train, y_train)

rf_predictions = rf_model.predict(X_test)

kfold = KFold(n_splits=10,random_state=7, shuffle=True)
scores = cross_val_score(rf_model, X_train, y_train, scoring='accuracy' ,cv=kfold).mean()

rf_train_acc = scores
print("RF training Accuracy of the mean of 10 - Fold Cross Validation is ", rf_train_acc)

rf_test_acc = accuracy_score(y_test, rf_predictions)
print("RF test Accuracy is ", rf_test_acc)

RF training Accuracy of the mean of 10 - Fold Cross Validation is  0.9672946859903382
RF test Accuracy is  0.9557522123893806
